# Analyze problem statement to identify approach to fulfill goals

Company faces significant challenges in optimizing crop yields and resource management <br>
Need to prioritize focus on these 2 objectives

1. Predict temperature conditions within farm's closed environment, ensuring optimal plant growth
  - Regression modelling task
  - Need to identify relevant/related features within provided database
<br><br>
2. Categorize combined "Plant Type-Stage" based on sensor data, aiding in strategic planning and resource allocation
  - Classification modelling task
  - Need to identify relevant/related features within provided database

# Setup environment, SQL connection and analyze SQL database

Necessary libraries will be imported when needed

Establish connection SQL database (agri.db) using relative path 'data/agri.db'

In [1]:
# Import libraries as needed
import sqlite3

# Set path to SQL database
db_path = "data/agri.db"

# Create connection to SQL database
conn = sqlite3.connect(db_path)

Set pandas options for better readability

In [2]:
import pandas as pd

pd.set_option('display.max_columns', None) # Display all columns in DataFrame
pd.set_option('display.max_rows', 100)     # Limit number of rows displayed to 100

Explore database structure by listing all tables to identify available tables for extraction

In [3]:
import pandas as pd

# Query to list all tables in database
query = "SELECT name FROM sqlite_master WHERE type='table';"
tables = pd.read_sql(query, conn)

# Display list of tables
tables

,name
0,farm_data


Since there is only 'farm_data' table in the database, the first few rows can be previewed to understand the column structure <br>
This can be cross-checked with the provided list of attributes in the PDF

In [4]:
# Preview first few rows of 'farm_data' table
farm_data_10_query = "SELECT * FROM farm_data LIMIT 10;"
farm_data_10_df = pd.read_sql(farm_data_10_query, conn)

# Display first few rows of table
farm_data_10_df.head(10)

,System Location Code,Previous Cycle Plant Type,Plant Type,Plant Stage,Temperature Sensor (°C),Humidity Sensor (%),Light Intensity Sensor (lux),CO2 Sensor (ppm),EC Sensor (dS/m),O2 Sensor (ppm),Nutrient N Sensor (ppm),Nutrient P Sensor (ppm),Nutrient K Sensor (ppm),pH Sensor,Water Level Sensor (mm)
0,Zone_D,Herbs,Vine Crops,Maturity,23.34,NaN,454.0,937,1.82,4,161,64,201,6.1,28.150346
1,Zone_G,Herbs,VINE CROPS,Vegetative,-24.16,63.14,441.0,938,2.13,6,161,53,None,6.5,31.000000
2,Zone_F,Herbs,herbs,maturity,23.84,NaN,562.0,1246,1.21,8,228,79,292,6.4,26.000000
3,Zone_G,Herbs,fruiting vegetables,Vegetative,22.61,NaN,651.0,1346,1.96,6,None,53 ppm,289,6.6,NaN
4,Zone_B,Vine Crops,LEAFY GREENS,seedling,22.88,NaN,144.0,812,2.76,5,61,19,168,5.5,28.000000
5,Zone_C,Vine Crops,FRUITING VEGETABLES,MATURITY,25.96,NaN,675.0,1338,1.75,8,227,53,258,6.2,23.092135
6,Zone_A,Leafy Greens,Vine Crops,Maturity,NaN,NaN,514.0,1103,1.14,8,174,62,217,5.5,35.000000
7,Zone_G,Herbs,Leafy Greens,Maturity,20.16,NaN,317.0,1133,1.52,10,None,None,277,5.8,39.611156
8,Zone_G,Herbs,Leafy Greens,Maturity,20.16,NaN,317.0,1133,1.52,10,None,None,277,5.8,39.611156
9,Zone_B,Vine Crops,Vine Crops,MATURITY,25.59,NaN,435.0,845,2.07,8,150,70,210,5.3,27.835062


There are a few issues in the current database that will have to be sorted out before the data can be used for feature engineering or used in machine learning modelling <br>
<br>
Currently identified issues:
| Column Name | Issue |
| :---------: | :---: |
| Plant Type  | Non-standardized naming format |
| Plant Stage | Non-standardized naming format |
| Temperature Sensor | Missing values (NaN) |
|                    | Negative value |
| Humidity Sensor | Missing values (NaN) |
| Nutrient * Sensor | Missing value (None) |
|                   | Values with units |
| Water Level Sensor | Missing value (NaN) |

The schema of 'farm_data' table can be retrieved to understand the columns' data type <br>
However, this might not be fully accurate prior to data clean-up due to missing or incorrectly labelled values

In [5]:
# Get schema of 'farm_data' table
schema_query = "PRAGMA table_info(farm_data);"
schema_df = pd.read_sql(schema_query, conn)

# Display schema information
schema_df

,cid,name,type,notnull,dflt_value,pk
0,0,System Location Code,TEXT,0,None,0
1,1,Previous Cycle Plant Type,TEXT,0,None,0
2,2,Plant Type,TEXT,0,None,0
3,3,Plant Stage,TEXT,0,None,0
4,4,Temperature Sensor (°C),REAL,0,None,0
5,5,Humidity Sensor (%),REAL,0,None,0
6,6,Light Intensity Sensor (lux),REAL,0,None,0
7,7,CO2 Sensor (ppm),INTEGER,0,None,0
8,8,EC Sensor (dS/m),REAL,0,None,0
9,9,O2 Sensor (ppm),INTEGER,0,None,0


Most of the columns' data type match expectations, except for Nutrient Sensors <br>
These columns should have REAL/INTEGER type but are currently of TEXT type <br> <br>
To handle this, the missing values will have to resolved and the values with units need to be processed <br>
After both steps are done, the columns' data can be converted to REAL/INTEGER type

# Perform Exploratory Data Analysis (EDA) on SQL table data

Load all data from 'farm_data' table into a DataFrame to start data analysis

In [6]:
# Get all data from 'farm_data' table
farm_data_query = "SELECT * FROM farm_data;"
farm_data_df = pd.read_sql_query(farm_data_query, conn)

Start with data preprocessing to clean-up missing values, non-standardized naming format and extra info in values

The data in columns (Plant Type, Plant Stage) will all be changed to lowercase characters to standardize the data

In [7]:
non_standard_name_list = ["Plant Type", "Plant Stage"]

for col_name in non_standard_name_list:
    farm_data_df[col_name] = farm_data_df[col_name].str.lower()


Remove 'ppm' from Nutrient * Sensor column data and convert the data into numeric type

In [9]:
ppm_drop_list = ["Nutrient N Sensor (ppm)", "Nutrient P Sensor (ppm)", "Nutrient K Sensor (ppm)"]

for col_name in ppm_drop_list:
    farm_data_df[col_name] = farm_data_df[col_name].str.replace("ppm", "", regex=False)
    farm_data_df[col_name] = pd.to_numeric(farm_data_df[col_name], errors="coerce")

Remove negative sign in Temperature Sensor column data

In [10]:
# Get Temperature Sensor column name
farm_data_df_col_list = farm_data_df.columns

for col_name in farm_data_df_col_list:
    if "Temperature Sensor" in col_name:
        temp_sensor_col_name = col_name

farm_data_df[temp_sensor_col_name] = farm_data_df[temp_sensor_col_name].abs()

After checking the current DataFrame, it seems that Light Intensity Sensor column also has missing values <br>
So that will be handled together with the other affected columns

For the remaining columns with missing values, they will be filled with either mean or median values of their zone <br>
The existing data of each column will be grouped into their System Location Code to obtain the mean and median values

Here is the breakdown of each column:
| Column Name | Mean/Median | Reason |
| :---------: | :---------: | :----: |
| Temperature Sensor | Mean | Data is relatively stable and has normal distribution |
| Humidity Sensor | Median  | Data is not normally distributed, no clear pattern |
| Light Intensity Sensor | Median | Data is skewed towards the upper half of the spectrum |
| Nutrient N Sensor | Median | Data has sudden dip and spike near right of spectrum |
| Nutrient P Sensor | Median | Data has sudden dip and spike near left of spectrum |
| Nutrient K Sensor | Median | Data has sudden dip and spike near right of spectrum |
| Water Level Sensor | Median | To avoid outliers at extreme ends of spectrum |